# Elasticsearch Inference API & Hugging Face

This notebook demonstrates how to use Hugging Face completions along with the Elasticsearch Inference API. This notebook is based on the article [Inference API & and Hugging Face](https://www.elastic.co/search-labs/blog/inference-api-and-hugging-face).

In [1]:
%pip install requests elasticsearch -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Installing dependencies and importing packages

In [2]:
import os
import json
import requests

from elasticsearch import Elasticsearch, helpers
from getpass import getpass

## Setting up environment variables

In [ ]:
os.environ["HUGGING_FACE_INFERENCE_ENDPOINT_URL"] = getpass(
    "Enter your Hugging Face Inference Endpoint URL: "
)
os.environ["ELASTICSEARCH_API_KEY"] = getpass("Enter your Elasticsearch API key: ")
os.environ["ELASTICSEARCH_URL"] = getpass("Enter your Elasticsearch URL: ")
os.environ["HUGGING_FACE_API_KEY"] = getpass("Enter your Hugging Face API key: ")


INDEX_NAME = "company-blog-posts"
INFERENCE_ENDPOINT_ID = "hugging-face-gpt-oss-safeguard"

## Elasticsearch Python client

In [4]:
es_client = Elasticsearch(
    os.environ["ELASTICSEARCH_URL"], api_key=os.environ["ELASTICSEARCH_API_KEY"]
)

## Hugging Face completions inference endpoint setup

Let’s create an Elasticsearch inference endpoint that uses the Hugging Face model. This endpoint will be used for chat completions, and this service will help us generate responses based on the dataset. To create this endpoint we’re using the [inference API](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-inference-put).

In [ ]:
try:
    resp = es_client.inference.put(
        task_type="chat_completion",
        inference_id=INFERENCE_ENDPOINT_ID,
        body={
            "service": "hugging_face",
            "service_settings": {
                "api_key": os.environ["HUGGING_FACE_API_KEY"],
                "url": os.environ["HUGGING_FACE_INFERENCE_ENDPOINT_URL"],
            },
        },
    )

    print(
        "Chat completion inference endpoint created successfully:", resp["inference_id"]
    )
except Exception as e:
    print("Error creating chat completion inference endpoint:", {e})

{'inference_id': 'hugging-face-gpt-oss-safeguard', 'task_type': 'chat_completion', 'service': 'hugging_face', 'service_settings': {'url': 'https://ibuf1kbe7xcak382.us-east-1.aws.endpoints.huggingface.cloud/v1/chat/completions', 'rate_limit': {'requests_per_minute': 3000}}}
Chat completion inference endpoint created successfully: hugging-face-gpt-oss-safeguard


## Index setup

### Creating mappings


In [23]:
try:
    mapping = {
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {
                    "type": "text",
                    "fields": {"keyword": {"type": "keyword"}},
                    "copy_to": "semantic_field",
                },
                "author": {"type": "keyword", "copy_to": "semantic_field"},
                "category": {"type": "keyword", "copy_to": "semantic_field"},
                "content": {"type": "text", "copy_to": "semantic_field"},
                "date": {"type": "date"},
                "semantic_field": {"type": "semantic_text"},
            }
        }
    }

    es_client.indices.create(index=INDEX_NAME, body=mapping)
    print(f"Index {INDEX_NAME} created successfully")
except Exception as e:
    print(f"Error creating index: {e}")

Index company-blog-posts created successfully


### Ingesting data to Elasticsearch

In [24]:
def build_data(json_file, index_name):
    with open(json_file, "r") as f:
        data = json.load(f)

    for doc in data:
        yield {"_index": index_name, "_source": doc}


try:
    success, errors = helpers.bulk(es_client, build_data("dataset.json", INDEX_NAME))
    print(f"{success} documents indexed successfully")

    if errors:
        print("Errors during indexing:", errors)
except Exception as e:
    print(f"Error: {str(e)}")

10 documents indexed successfully


## Function to execute semantic search

In [11]:
def semantic_search(user_question: str, size: int = 5):
    try:
        response = es_client.search(
            index=INDEX_NAME,
            body={
                "query": {"match": {"semantic_field": user_question}},
                "size": size,
            },
        )

        return {
            "hits": response["hits"]["hits"],
            "total_hits": len(response["hits"]["hits"]),
        }

    except Exception as e:
        print(f"Error searching index: {str(e)}")

In [33]:
results = semantic_search(
    user_question="Search for updates related to data encryption.", size=1
)

print(f"Total hits: {results['total_hits']}")
for hit in results["hits"]:
    print(f"{json.dumps(hit['_source'], indent=2)}")

Total hits: 1
{
  "id": "8",
  "title": "Security update: encrypted data pipelines",
  "author": "Daniel Vega",
  "date": "2025-11-08",
  "category": "security",
  "content": "We have enhanced our data pipelines with full encryption in transit and at rest. This aligns with our company\u2019s commitment to protecting customer data."
}


## Generating completions function

In [22]:
def stream_chat_completion(messages: list, inference_id: str = INFERENCE_ENDPOINT_ID):
    url = f"{os.environ['ELASTICSEARCH_URL']}/_inference/chat_completion/{inference_id}/_stream"

    payload = {"messages": messages}

    ELASTICSEARCH_HEADERS = {
        "Authorization": f"ApiKey {os.environ['ELASTICSEARCH_API_KEY']}",
        "Content-Type": "application/json",
    }

    try:
        response = requests.post(
            url, json=payload, headers=ELASTICSEARCH_HEADERS, stream=True
        )
        response.raise_for_status()

        for line in response.iter_lines(decode_unicode=True):
            if line:
                line = line.strip()

                # Skip event lines like "event: message"
                if line.startswith("event:"):
                    continue

                # Process data lines
                if line.startswith("data: "):
                    data_content = line[6:]  # Remove "data: " prefix

                    # Skip empty data or special markers
                    if not data_content.strip() or data_content.strip() == "[DONE]":
                        continue

                    try:
                        chunk_data = json.loads(data_content)

                        # Extract the content from the Mistral response structure
                        if "choices" in chunk_data and len(chunk_data["choices"]) > 0:
                            choice = chunk_data["choices"][0]
                            if "delta" in choice and "content" in choice["delta"]:
                                content = choice["delta"]["content"]
                                if content:  # Only yield non-empty content
                                    yield content

                    except json.JSONDecodeError as json_err:
                        # If JSON parsing fails, log the error but continue
                        print(f"\nJSON decode error: {json_err}")
                        print(f"Problematic data: {data_content}")
                        continue

    except requests.exceptions.RequestException as e:
        yield f"Error: {str(e)}"

## RAG Chat with Streaming


In [ ]:
def moderation_chat(
    user_query: str,
    inference_id: str = INFERENCE_ENDPOINT_ID,
    ruleset_path: str = "policies.txt",
):
    search_results = semantic_search(user_query)
    context_docs = search_results["hits"]

    try:
        with open(ruleset_path, "r", encoding="utf-8") as f:
            ruleset = f.read()
    except FileNotFoundError:
        raise FileNotFoundError(
            "ruleset.txt not found. Please provide the company policies file."
        )

    system_prompt = f"""
        You are a Safeguard moderation assistant.
        Evaluate the provided article according to the company content policies.

        The evaluation must follow these instructions:
        - Only use the provided ruleset to judge compliance.
        - Focus on tone, confidentiality, and adherence to internal communication standards.

        Company ruleset:
        {ruleset}
    """

    results = []

    for i, doc in enumerate(context_docs, 1):
        title = doc["_source"].get("title", f"Article {i}")
        content = doc["_source"].get("content", "")

        user_prompt = f"""
            Evaluate this article:
            Title: {title}
            Content body:
            {content}
        """

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]

        print("\n------------------------------------------------")
        print(
            f"Evaluating article {i}/{len(context_docs)}\n TITLE: {title} \n CONTENT: {content}...\n"
        )

        try:
            full_response = ""
            for chunk in stream_chat_completion(messages, inference_id=inference_id):
                print(chunk, end="", flush=True)
                full_response += chunk

            results.append({"title": title, "response": full_response.strip()})
        except Exception as e:
            print(f"\n⚠️ Error evaluating {title}: {e}")
            results.append({"title": title, "response": None, "error": str(e)})

    return results

In [24]:
response = moderation_chat(
    user_query="Find posts explaining debugging or authentication issues"
)
# Find posts explaining debugging or authentication issues.

# “Find articles about analytics dashboards or visualization tools.”

[{'_index': 'company-blog-posts', '_id': 'Xp1ngpoB2AKc8jP6brQU', '_score': 32.036026, '_source': {'id': '6', 'title': 'Debugging authentication issues', 'author': 'Tomás Hernández', 'date': '2025-11-06', 'category': 'engineering', 'content': 'When debugging authentication, never share production tokens or client credentials. Always use our test environment and anonymized data for public examples.'}}, {'_index': 'company-blog-posts', '_id': 'X51ngpoB2AKc8jP6brQU', '_score': 5.9653068, '_source': {'id': '7', 'title': 'Meet our competitors head-on!', 'author': 'Alice Walker', 'date': '2025-11-07', 'category': 'marketing', 'content': 'Unlike BrandX, our software is actually stable and bug-free. This article compares us directly to competitors and mocks their quality issues.'}}, {'_index': 'company-blog-posts', '_id': 'YJ1ngpoB2AKc8jP6brQU', '_score': 4.9244947, '_source': {'id': '8', 'title': 'Security update: encrypted data pipelines', 'author': 'Daniel Vega', 'date': '2025-11-08', 'categ

In [ ]:
response = moderation_chat(
    "Find posts explaining debugging or authentication issues",
    inference_id="gpt-oss-20b-endpoint",
)

## Deleting

Delete the resources used to prevent them from consuming resources.

In [22]:
# Cleanup - Delete Index
es_client.indices.delete(index=INDEX_NAME)
# TODO: delete inference

ObjectApiResponse({'acknowledged': True})